In [ ]:
# %run ./fyers_auth.ipynb

In [ ]:
from tensorflow import keras
import sys
sys.path.append('./shared')
sys.path.append('./strategies')
sys.path.append('./sql')
from data_fetching_func import dynamicDataFetching,clean_data 
from ema_crossover import ema_crossover
from money_management_rules import already_in_position,check_three_loses
import pandas as pd
import pandas_ta as ta
import datetime as dt
import numpy as np
from sql_config import connect_db,execute_and_fetch

In [ ]:
db = connect_db()
cursor = db.cursor()
cursor.execute("USE tradingbot")

Importing Model

# Trend Prediction

In [ ]:
startDate = (dt.date.today() - dt.timedelta(days = 800)) #last working day
endDate = dt.date.today()
resolution = "1D"
trend_df = dynamicDataFetching(fyers,startDate,endDate,symbol,resolution)
clean_data(trend_df)

In [ ]:
trend_df.shape

In [ ]:
trend_df.tail()

In [ ]:
# Adding indicators
trend_df['RSI']=ta.rsi(trend_df.close, length=15)
trend_df['EMA_20']=ta.ema(trend_df.close, length=20)
trend_df['EMA_100']=ta.ema(trend_df.close, length=100)
trend_df['EMA_150']=ta.ema(trend_df.close, length=150)
trend_df['EMA_200']=ta.ema(trend_df.close, length=200)
trend_df['EMA_500']=ta.ema(trend_df.close, length=500)


# Dropping null values after adding indicators and features
trend_df.dropna(inplace=True)
trend_df.reset_index(inplace = True)
# trend_df.drop(['volume', 'date'], axis=1, inplace=True) # not required
trend_df.drop([ 'date'], axis=1, inplace=True) # not required

In [ ]:
trend_df = trend_df.tail(30)
trend_df.tail()


In [ ]:
trend_df.shape

In [ ]:
# Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(trend_df)
print(data_set_scaled)

In [ ]:
data_set_scaled = np.reshape(data_set_scaled, (1, 30, 11))

In [ ]:
data_set_scaled.shape

In [ ]:
model = keras.models.load_model('./models/lstm_trend_1.0/')
model.summary()

In [ ]:
y_pred = model.predict(data_set_scaled)
y_pred.shape

In [ ]:
trend_signal = int(y_pred)
if trend_signal == 0 :
    trend_signal = -1 #DOWN TREND
elif trend_signal == 1:
    trend_signal = 1 #UP TREND

cursor.execute(f"INSERT INTO trend VALUES(curdate(),'{symbol}',{trend_signal})")
db.commit()
# TODO - store in some kind of db then retrieve afterwards when required

# Techinal Strategy

Data fetching for techinal indicator


In [ ]:

startDate = (dt.date.today() - dt.timedelta(days = 1)) #if week day

if  dt.date.today().weekday() == 6: #if sunday
    startDate = (dt.date.today() - dt.timedelta(days = 2)) 
if dt.date.today().weekday() == 0: #if monday
    startDate = (dt.date.today() - dt.timedelta(days = 3)) 

endDate = dt.date.today()
# symbol="NSE:TATASTEEL-EQ"
resolution = "5"
technical_df = dynamicDataFetching(fyers,startDate,endDate,symbol,resolution)
clean_data(technical_df)
technical_df = technical_df.rename(columns={"open":"Open","low":"Low","high":"High","close":"Close","volume":"Volume"})

In [ ]:
technical_df.shape

In [ ]:
technical_signal = ema_crossover(technical_df)

Now we have both trend_signal and technical_signal, we can have final singal

In [ ]:

final_signal = 0
if trend_signal == 2 and technical_signal == 2:
  final_signal = 1 #BUY
elif trend_signal == 1 and technical_signal == 1:
  final_signal = -1 #SELL

In [ ]:

positions = fyers.positions()
#  check if already in position
#  check if done for the day or not
can_trade = (already_in_position(positions)==False) and (check_three_loses(positions)==False)

# Taking trades
if can_trade & final_signal != 0:
    stop_loss = 2*technical_df.ATR[-1]
    target = 2*stop_loss
    data = {
        "symbol":symbol,
        "qty":1,
        "type":2,
        "side":final_signal,
        "productType":"BO",
        "limitPrice":0,
        "stopPrice":0,
        "validity":"DAY",
        "disclosedQty":0,
        "offlineOrder":"False",
        "stopLoss":stop_loss,
        "takeProfit":target
    }
    resp = fyers.place_order(data)
    # if resp["code"] == 1101:
    print("Order placed successfully")
    curr_price = fyers.quotes({"symbols":symbol})
    curr_price = curr_price["d"][0]["v"]["lp"]
    absolute_target = curr_price 
    absolute_sl = curr_price
    if final_signal == 1:
        absolute_target = curr_price+target
        absolute_sl = curr_price-target
    elif final_signal == -1:
        absolute_target = curr_price-target
        absolute_sl = curr_price+target
    
    # storing in db 
    cursor.execute(f"""INSERT INTO tradebook VALUES
    (NOW(),'{symbol}',{curr_price},{absolute_target},{absolute_sl},{final_signal})
    """)
    db.commit()
    # else:
    #     print("Order not placed successfully")
else :
    print("No trade - can_trade & final_signal")